In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import re

In [3]:
train=pd.read_csv('/content/drive/MyDrive/Hackathon/Workation Price Prediction Challenge/Train.csv')
test=pd.read_csv('/content/drive/MyDrive/Hackathon/Workation Price Prediction Challenge/Test.csv')

In [4]:
tests=test.copy()

In [5]:
train.head()

,Uniq Id,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price
0,e788ab76d9d8cf1e6ed2f139645ca5d1,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,New Delhi|Shimla|Manali|Chandigarh,30-07-2021,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0
1,178f892630ce3e335a5a41d5d83937fd,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,Srinagar|Pahalgam|Srinagar,08-12-2021,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5
2,f060f2954840503cc2fdaf495357b7df,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,Udaipur|Chittorgarh,26-04-2021,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...,12421.5
3,32a19a6c171e67448f2346da46c619dc,Colorful Kerala ( Romantic Getaway ),Premium,Munnar|Kumarakom|Allepey|Kovalam and Poovar,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Munnar|Kumarakom|Allepey|Kovalam and Poovar,27-08-2021,Elixir Hills Suites Resort & Spa-MMT Holidays ...,New Delhi,IndiGo,0,5,Mattupetty Dam | Echo Point | Tata Tea Museum...,Cancellation any time after making the 1st pay...,35967.0
4,107b068aa0ca03bc6248966f594d105f,A Week In Bangkok & Pattaya,Premium,Pattaya|Bangkok,4N Pattaya . 3N Bangkok,Pattaya|Bangkok,12-12-2021,Dusit Thani Pattaya - MMT Special:4.5|Amari Wa...,New Delhi,Spicejet|Go Air,0,5,"Coral Island Tour with Indian Lunch, Join Spe...",Cancellation any time after making the 1st pay...,25584.0


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Uniq Id                     21000 non-null  object 
 1   Package Name                21000 non-null  object 
 2   Package Type                21000 non-null  object 
 3   Destination                 21000 non-null  object 
 4   Itinerary                   21000 non-null  object 
 5   Places Covered              21000 non-null  object 
 6   Travel Date                 21000 non-null  object 
 7   Hotel Details               21000 non-null  object 
 8   Start City                  21000 non-null  object 
 9   Airline                     21000 non-null  object 
 10  Flight Stops                21000 non-null  int64  
 11  Meals                       21000 non-null  int64  
 12  Sightseeing Places Covered  21000 non-null  object 
 13  Cancellation Rules          210

In [7]:
train.isna().sum()

Uniq Id                       0
Package Name                  0
Package Type                  0
Destination                   0
Itinerary                     0
Places Covered                0
Travel Date                   0
Hotel Details                 0
Start City                    0
Airline                       0
Flight Stops                  0
Meals                         0
Sightseeing Places Covered    0
Cancellation Rules            0
Per Person Price              0
dtype: int64

In [8]:
train.drop(['Uniq Id','Places Covered'],axis=1,inplace=True)
test.drop(['Uniq Id','Places Covered'],axis=1,inplace=True)

In [9]:
train.head(3)

,Package Name,Package Type,Destination,Itinerary,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price
0,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,30-07-2021,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0
1,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,08-12-2021,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5
2,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,26-04-2021,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...,12421.5


In [10]:
train['Travel Date']=pd.to_datetime(train['Travel Date'])
test['Travel Date']=pd.to_datetime(test['Travel Date'])

In [11]:
train['Travel Year']=train['Travel Date'].dt.year
train['Travel Month']=train['Travel Date'].dt.month

In [12]:
test['Travel Year']=test['Travel Date'].dt.year
test['Travel Month']=test['Travel Date'].dt.month

In [13]:
train.drop(['Travel Date'],axis=1,inplace=True)
test.drop(['Travel Date'],axis=1,inplace=True)

In [14]:
train.head(3)

,Package Name,Package Type,Destination,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Travel Year,Travel Month
0,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0,2021,7
1,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,The Orchard Retreat & Spa:4.6|WelcomHotel Pine...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5,2021,8
2,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,The Ananta:4.4|juSTa Lake Nahargarh Palace:4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...,12421.5,2021,4


In [15]:
train['Total Destination Covered']=train['Destination'].apply(lambda x: len(x.split('|')))
test['Total Destination Covered']=test['Destination'].apply(lambda x: len(x.split('|')))

In [16]:
train['Total Sightseeing Places Covered']=train['Sightseeing Places Covered'].apply(lambda x: len(x.split('|')) if x!='Not Available' else -1)
test['Total Sightseeing Places Covered']=test['Sightseeing Places Covered'].apply(lambda x: len(x.split('|')) if x!='Not Available' else -1)

In [17]:
print(train['Itinerary'][0])
print(train['Itinerary'][1])

1N New Delhi . 2N Shimla . 2N Manali . 1N Chandigarh
1N Srinagar . 2N Pahalgam . 1N Srinagar


In [18]:
train['Total Nights Stay']=train['Itinerary'].apply(lambda x: sum(int(i) for i in re.findall(r'[0-9]',x)))
test['Total Nights Stay']=test['Itinerary'].apply(lambda x: sum(int(i) for i in re.findall(r'[0-9]',x)))

In [19]:
print(train['Hotel Details'][1])
print(train['Hotel Details'][2])

The Orchard Retreat & Spa:4.6|WelcomHotel Pine N Peak Pahalgam - Member ITC Hotel Group:4.1|The Orchard Retreat & Spa:4.6
The Ananta:4.4|juSTa Lake Nahargarh Palace:4


In [20]:
train['Hotel Details']=train['Hotel Details'].str.replace(':','')
test['Hotel Details']=test['Hotel Details'].str.replace(':','')

In [21]:
train['Hotel Rating']=train['Hotel Details'].apply(lambda x:sum(float(i) for i in re.findall(r'[0-9]*[.][0-9]',x))/len(re.findall(r'[0-9]*[.][0-9]',x)) if len(re.findall(r'[0-9]*[.][0-9]',x)) !=0 else -1)

In [22]:
test['Hotel Rating']=test['Hotel Details'].apply(lambda x:sum(float(i) for i in re.findall(r'[0-9]*[.][0-9]',x))/len(re.findall(r'[0-9]*[.][0-9]',x)) if len(re.findall(r'[0-9]*[.][0-9]',x)) !=0 else -1)

In [23]:
train.head(3)

,Package Name,Package Type,Destination,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Travel Year,Travel Month,Total Destination Covered,Total Sightseeing Places Covered,Total Nights Stay,Hotel Rating
0,Best of Shimla and Manali Holiday from Delhi,Standard,New Delhi|Shimla|Manali|Chandigarh,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,Not Available,Mumbai,Not Available,2,3,Not Available,Not Available,11509.0,2021,7,4,-1,6,-1.000000
1,Kashmir Valley vacation,Premium,Srinagar|Pahalgam|Srinagar,1N Srinagar . 2N Pahalgam . 1N Srinagar,The Orchard Retreat & Spa4.6|WelcomHotel Pine ...,New Delhi,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,Cancellation any time after making the 1st pay...,22485.5,2021,8,3,7,4,4.433333
2,Might of Mewar- Udaipur and Chittorgarh,Luxury,Udaipur|Chittorgarh,2N Udaipur . 1N Chittorgarh,The Ananta4.4|juSTa Lake Nahargarh Palace4,New Delhi,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,Cancellation any time after making the 1st pay...,12421.5,2021,4,2,8,3,4.400000


In [24]:
train['Sightseeing Places Covered in one day']=train['Total Sightseeing Places Covered']/train['Total Nights Stay']
test['Sightseeing Places Covered in one day']=test['Total Sightseeing Places Covered']/test['Total Nights Stay']

In [25]:
train['Total Destination Covered in one day']=train['Total Destination Covered']/train['Total Nights Stay']
test['Total Destination Covered in one day']=test['Total Destination Covered']/test['Total Nights Stay']

In [26]:
ordinal_columns=['Package Name','Destination','Start City','Cancellation Rules']

In [27]:
tfidf_columns=['Package Type','Itinerary','Hotel Details','Airline','Sightseeing Places Covered']

In [28]:
oe=OrdinalEncoder()
def oe_trans(df,column):
  df[column]=oe.fit_transform(df[column])
  return df

In [29]:
train=oe_trans(train,ordinal_columns)
test=oe_trans(test,ordinal_columns)

In [30]:
train.head()

,Package Name,Package Type,Destination,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Travel Year,Travel Month,Total Destination Covered,Total Sightseeing Places Covered,Total Nights Stay,Hotel Rating,Sightseeing Places Covered in one day,Total Destination Covered in one day
0,421.0,Standard,417.0,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,Not Available,0.0,Not Available,2,3,Not Available,5.0,11509.0,2021,7,4,-1,6,-1.000000,-0.166667,0.666667
1,1138.0,Premium,533.0,1N Srinagar . 2N Pahalgam . 1N Srinagar,The Orchard Retreat & Spa4.6|WelcomHotel Pine ...,1.0,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,0.0,22485.5,2021,8,3,7,4,4.433333,1.750000,0.750000
2,1417.0,Luxury,549.0,2N Udaipur . 1N Chittorgarh,The Ananta4.4|juSTa Lake Nahargarh Palace4,1.0,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,0.0,12421.5,2021,4,2,8,3,4.400000,2.666667,0.666667
3,486.0,Premium,329.0,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Elixir Hills Suites Resort & Spa-MMT Holidays ...,1.0,IndiGo,0,5,Mattupetty Dam | Echo Point | Tata Tea Museum...,0.0,35967.0,2021,8,4,9,6,4.500000,1.500000,0.666667
4,196.0,Premium,443.0,4N Pattaya . 3N Bangkok,Dusit Thani Pattaya - MMT Special4.5|Amari Wat...,1.0,Spicejet|Go Air,0,5,"Coral Island Tour with Indian Lunch, Join Spe...",0.0,25584.0,2021,12,2,2,7,4.350000,0.285714,0.285714


In [31]:
def TfidfVect(column, train, test):
    vect = TfidfVectorizer()
    
    train_feature = vect.fit_transform(train[column])
    train_feature = pd.DataFrame(train_feature.todense(), columns=[(column+str(i)) for i in range(len(vect.get_feature_names()))])
    train = pd.concat([train, train_feature], axis=1)
    
    test_feature = vect.transform(test[column])
    test_feature = pd.DataFrame(test_feature.todense(), columns=[(column+str(i)) for i in range(len(vect.get_feature_names()))])
    test = pd.concat([test, test_feature], axis=1)
    
    yield train, test

In [32]:
for column in tfidf_columns:
    train, test = next(TfidfVect(column, train, test))

In [33]:
train

,Package Name,Package Type,Destination,Itinerary,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Per Person Price,Travel Year,Travel Month,Total Destination Covered,Total Sightseeing Places Covered,Total Nights Stay,Hotel Rating,Sightseeing Places Covered in one day,Total Destination Covered in one day,Package Type0,Package Type1,Package Type2,Package Type3,Package Type4,Itinerary0,Itinerary1,Itinerary2,Itinerary3,Itinerary4,Itinerary5,Itinerary6,Itinerary7,Itinerary8,Itinerary9,Itinerary10,Itinerary11,Itinerary12,Itinerary13,Itinerary14,...,Sightseeing Places Covered1398,Sightseeing Places Covered1399,Sightseeing Places Covered1400,Sightseeing Places Covered1401,Sightseeing Places Covered1402,Sightseeing Places Covered1403,Sightseeing Places Covered1404,Sightseeing Places Covered1405,Sightseeing Places Covered1406,Sightseeing Places Covered1407,Sightseeing Places Covered1408,Sightseeing Places Covered1409,Sightseeing Places Covered1410,Sightseeing Places Covered1411,Sightseeing Places Covered1412,Sightseeing Places Covered1413,Sightseeing Places Covered1414,Sightseeing Places Covered1415,Sightseeing Places Covered1416,Sightseeing Places Covered1417,Sightseeing Places Covered1418,Sightseeing Places Covered1419,Sightseeing Places Covered1420,Sightseeing Places Covered1421,Sightseeing Places Covered1422,Sightseeing Places Covered1423,Sightseeing Places Covered1424,Sightseeing Places Covered1425,Sightseeing Places Covered1426,Sightseeing Places Covered1427,Sightseeing Places Covered1428,Sightseeing Places Covered1429,Sightseeing Places Covered1430,Sightseeing Places Covered1431,Sightseeing Places Covered1432,Sightseeing Places Covered1433,Sightseeing Places Covered1434,Sightseeing Places Covered1435,Sightseeing Places Covered1436,Sightseeing Places Covered1437
0,421.0,Standard,417.0,1N New Delhi . 2N Shimla . 2N Manali . 1N Chan...,Not Available,0.0,Not Available,2,3,Not Available,5.0,11509.0,2021,7,4,-1,6,-1.000000,-0.166667,0.666667,0.0,0.0,0.0,0.0,1.0,0.356034,0.294892,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,1138.0,Premium,533.0,1N Srinagar . 2N Pahalgam . 1N Srinagar,The Orchard Retreat & Spa4.6|WelcomHotel Pine ...,1.0,IndiGo|IndiGo,0,5,Dal Lake | Avantipura Ruins | Mughal Gardens ...,0.0,22485.5,2021,8,3,7,4,4.433333,1.750000,0.750000,0.0,0.0,0.0,1.0,0.0,0.342959,0.142031,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,1417.0,Luxury,549.0,2N Udaipur . 1N Chittorgarh,The Ananta4.4|juSTa Lake Nahargarh Palace4,1.0,IndiGo,0,4,Lake Pichola | Jag Mandir Palace | Saheliyon ...,0.0,12421.5,2021,4,2,8,3,4.400000,2.666667,0.666667,0.0,0.0,1.0,0.0,0.0,0.203781,0.168785,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,486.0,Premium,329.0,2N Munnar . 1N Kumarakom . 1N Allepey . 2N Kov...,Elixir Hills Suites Resort & Spa-MMT Holidays ...,1.0,IndiGo,0,5,Mattupetty Dam | Echo Point | Tata Tea Museum...,0.0,35967.0,2021,8,4,9,6,4.500000,1.500000,0.666667,0.0,0.0,0.0,1.0,0.0,0.303143,0.251084,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.197673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,196.0,Premium,443.0,4N Pattaya . 3N Bangkok,Dusit Thani Pattaya - MMT Special4.5|Amari Wat...,1.0,Spicejet|Go Air,0,5,"Coral Island Tour with I

In [34]:
train.drop(tfidf_columns,axis=1,inplace=True)
test.drop(tfidf_columns,axis=1,inplace=True)

In [35]:
from xgboost import XGBRegressor

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X=train.drop(['Per Person Price'],axis=1)
y=train['Per Person Price']

In [38]:
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X

,Package Name,Destination,Start City,Flight Stops,Meals,Cancellation Rules,Travel Year,Travel Month,Total Destination Covered,Total Sightseeing Places Covered,Total Nights Stay,Hotel Rating,Sightseeing Places Covered in one day,Total Destination Covered in one day,Package Type0,Package Type1,Package Type2,Package Type3,Package Type4,Itinerary0,Itinerary1,Itinerary2,Itinerary3,Itinerary4,Itinerary5,Itinerary6,Itinerary7,Itinerary8,Itinerary9,Itinerary10,Itinerary11,Itinerary12,Itinerary13,Itinerary14,Itinerary15,Itinerary16,Itinerary17,Itinerary18,Itinerary19,Itinerary20,...,Sightseeing Places Covered1398,Sightseeing Places Covered1399,Sightseeing Places Covered1400,Sightseeing Places Covered1401,Sightseeing Places Covered1402,Sightseeing Places Covered1403,Sightseeing Places Covered1404,Sightseeing Places Covered1405,Sightseeing Places Covered1406,Sightseeing Places Covered1407,Sightseeing Places Covered1408,Sightseeing Places Covered1409,Sightseeing Places Covered1410,Sightseeing Places Covered1411,Sightseeing Places Covered1412,Sightseeing Places Covered1413,Sightseeing Places Covered1414,Sightseeing Places Covered1415,Sightseeing Places Covered1416,Sightseeing Places Covered1417,Sightseeing Places Covered1418,Sightseeing Places Covered1419,Sightseeing Places Covered1420,Sightseeing Places Covered1421,Sightseeing Places Covered1422,Sightseeing Places Covered1423,Sightseeing Places Covered1424,Sightseeing Places Covered1425,Sightseeing Places Covered1426,Sightseeing Places Covered1427,Sightseeing Places Covered1428,Sightseeing Places Covered1429,Sightseeing Places Covered1430,Sightseeing Places Covered1431,Sightseeing Places Covered1432,Sightseeing Places Covered1433,Sightseeing Places Covered1434,Sightseeing Places Covered1435,Sightseeing Places Covered1436,Sightseeing Places Covered1437
0,-1.002439,0.761160,-1.374691,1.353001,-0.744652,1.207148,-0.205638,0.155492,1.116925,-1.691008,0.709795,-0.114913,-1.652254,0.698559,-0.150646,-0.632898,-0.430079,-0.601628,1.644493,1.133412,0.684090,-0.648013,-0.251206,-0.136223,-0.098652,-0.051135,-0.037823,-0.140833,-0.164237,-0.035513,-0.016775,-0.054462,-0.022893,-0.015432,-0.292266,-0.036024,-0.009759,-0.160823,-0.196872,-0.037783,...,-0.155686,-0.044246,-0.011953,-0.177775,-0.074353,-0.022728,-0.016906,-0.009759,-0.039464,-0.207179,-0.221980,-0.079799,-0.025828,-0.026354,-0.02929,-0.016906,-0.024888,-0.200633,-0.021827,-0.013803,-0.051218,-0.336436,-0.145587,-0.062135,-0.018123,-0.01826,-0.009759,-0.046415,-0.063464,-0.016906,-0.022893,-0.01826,-0.030733,-0.025828,-0.122633,-0.026694,-0.037544,-0.087688,-0.304886,-0.226309
1,0.125036,1.487340,0.727436,-1.042296,1.493461,-0.430075,-0.205638,0.464797,0.344450,-0.076359,-0.408034,-0.109443,0.129924,1.155833,-0.150646,-0.632898,-0.430079,1.662157,-0.608090,1.057257,-0.322246,-0.648013,-0.251206,-0.136223,-0.098652,-0.051135,-0.037823,-0.140833,-0.164237,-0.035513,-0.016775,-0.054462,-0.022893,-0.015432,-0.292266,-0.036024,-0.009759,-0.160823,-0.196872,-0.037783,...,-0.155686,-0.044246,-0.011953,-0.177775,-0.074353,-0.022728,-0.016906,-0.009759,-0.039464,-0.207179,-0.221980,-0.079799,-0.025828,-0.026354,-0.02929,-0.016906,-0.024888,-0.200633,-0.021827,-0.013803,-0.051218,-0.336436,-0.145587,-0.062135,-0.018123,-0.01826,-0.009759,-0.046415,-0.063464,-0.016906,-0.022893,-0.01826,-0.030733,-0.025828,-0.122633,-0.026694,-0.037544,-0.087688,-0.304886,-0.226309
2,0.563760,1.587503,0.727436,-1.042296,0.374404,-0.430075,-0.205638,-0.772422,-0.428025,0.125472,-0.966949,-0.109477,0.982270,0.698559,-0.150646,-0.632898,2.325155,-0.601628,-0.608090,0.246633,-0.146114,-0.648013,-0.251206,-0.136223,-0.098652,-0.051135,-0.037823,-0.140833,-0.164237,-0.035513,-0.016775,-0.054462,-0.022893,-0.015432,-0.292266,-0.036024,-0.009759,-0.160823,-0.196872,-0.037783,...,-0.155686,-0.044246,-0.011953,-0.177775,-0.074353,-0.022728,-0.016906,-0.009759,-0.039464,-0.207179,-0.221980,-0.079799,-0.025828,-0.026354,-0.02929,-0.016906,-0.024888,-0.200633,-0.021827,-0.0138

In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,np.log(y),test_size=0.3,random_state=42)

In [40]:
from lightgbm import LGBMRegressor

In [41]:
lgb=LGBMRegressor()
lgb.fit(X_train,y_train)

LGBMRegressor()

In [42]:
lgb.score(X_test,y_test)

0.8923212844159821

In [43]:
y_pred=lgb.predict(X_test)

In [44]:
preds=pd.DataFrame()
preds['real_price']=np.expm1(y_test)
preds['predicted_by_model']=np.expm1(y_pred)
preds

,real_price,predicted_by_model
2312,20898.5,20952.964363
16845,13768.5,16706.303065
6105,29866.0,28110.011265
18082,12628.5,13740.484687
11335,25511.0,28828.603092
...,...,...
2562,26370.0,24840.104439
19682,12075.5,11705.342989
19988,15235.0,18045.537953
3198,7309.0,7638.247476


In [45]:
mse=mean_squared_error(y_pred,y_test)

In [46]:
mse

0.031869480179613684

In [47]:
test

,Package Name,Destination,Start City,Flight Stops,Meals,Cancellation Rules,Travel Year,Travel Month,Total Destination Covered,Total Sightseeing Places Covered,Total Nights Stay,Hotel Rating,Sightseeing Places Covered in one day,Total Destination Covered in one day,Package Type0,Package Type1,Package Type2,Package Type3,Package Type4,Itinerary0,Itinerary1,Itinerary2,Itinerary3,Itinerary4,Itinerary5,Itinerary6,Itinerary7,Itinerary8,Itinerary9,Itinerary10,Itinerary11,Itinerary12,Itinerary13,Itinerary14,Itinerary15,Itinerary16,Itinerary17,Itinerary18,Itinerary19,Itinerary20,...,Sightseeing Places Covered1398,Sightseeing Places Covered1399,Sightseeing Places Covered1400,Sightseeing Places Covered1401,Sightseeing Places Covered1402,Sightseeing Places Covered1403,Sightseeing Places Covered1404,Sightseeing Places Covered1405,Sightseeing Places Covered1406,Sightseeing Places Covered1407,Sightseeing Places Covered1408,Sightseeing Places Covered1409,Sightseeing Places Covered1410,Sightseeing Places Covered1411,Sightseeing Places Covered1412,Sightseeing Places Covered1413,Sightseeing Places Covered1414,Sightseeing Places Covered1415,Sightseeing Places Covered1416,Sightseeing Places Covered1417,Sightseeing Places Covered1418,Sightseeing Places Covered1419,Sightseeing Places Covered1420,Sightseeing Places Covered1421,Sightseeing Places Covered1422,Sightseeing Places Covered1423,Sightseeing Places Covered1424,Sightseeing Places Covered1425,Sightseeing Places Covered1426,Sightseeing Places Covered1427,Sightseeing Places Covered1428,Sightseeing Places Covered1429,Sightseeing Places Covered1430,Sightseeing Places Covered1431,Sightseeing Places Covered1432,Sightseeing Places Covered1433,Sightseeing Places Covered1434,Sightseeing Places Covered1435,Sightseeing Places Covered1436,Sightseeing Places Covered1437
0,797.0,177.0,1.0,0,4,0.0,2021,7,1,-1,5,4.100,-0.200000,0.200000,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.776319,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003.0,39.0,1.0,0,4,6.0,2021,3,1,1,2,4.400,0.500000,0.500000,0.0,0.0,1.0,0.0,0.0,0.000000,0.203108,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1972.0,314.0,1.0,2,3,6.0,2022,9,1,4,3,-1.000,1.333333,0.333333,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.559996,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.268419,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.203466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1359.0,314.0,0.0,1,3,6.0,2021,5,1,6,3,-1.000,2.000000,0.333333,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,0.559996,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1798.0,87.0,1.0,0,4,0.0,2021,10,4,10,5,4.550,2.000000,0.800000,0.0,0.0,1.0,0.0,0.0,0.555575,0.153388,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.393819,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.327193,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [48]:
test_pred=lgb.predict(test)

In [49]:
test_pred

array([10.62685267, 10.41013083,  9.76570054, ..., 10.48957395,
       10.25372696,  9.96096995])

In [50]:
tests['Price']=np.expm1(test_pred)

In [51]:
tests

,Uniq Id,Package Name,Package Type,Destination,Itinerary,Places Covered,Travel Date,Hotel Details,Start City,Airline,Flight Stops,Meals,Sightseeing Places Covered,Cancellation Rules,Price
0,12c47aefbcca65d6156ba0bc7946704b,Goa 5N stay with flights & Transfers-North Goa,Luxury,Goa,5N Goa,Goa,13-07-2021,Novotel Goa Resort & Spa:4.1,New Delhi,IndiGo,0,4,Not Available,Cancellation any time after making the 1st pay...,41226.165125
1,156febf7d27315ae9f210ed13bca3ac2,Into the Wild - Weekend at Bandipur National Park,Luxury,Bandipur,2N Bandipur,Bandipur,20-03-2021,The Windflower Jungle Resorts & Spa Bandipur:4.4,New Delhi,Not Available,0,4,Bandipur National Park,This cancellation policy is applicable for boo...,33193.212842
2,a7f43314e3ee6464a4e35b874bddf2ea,Unforgettable Munnar Getaway - Private Picnic ...,Standard,Munnar,3N Munnar,Munnar,09-02-2022,Not Available,New Delhi,Not Available,2,3,Old Munnar Town | Eravikulam national park | ...,This cancellation policy is applicable for boo...,17424.684977
3,d9cc4b407f3d5d290560c03d9ac71035,Munnar for Adventure Lovers - Top Valley Trek,Deluxe,Munnar,3N Munnar,Munnar,16-05-2021,Not Available,Mumbai,Not Available,1,3,Tata Tea Museum | Eravikulam national park | ...,This cancellation policy is applicable for boo...,17608.746902
4,b3bbc8aa6429feb260e9ac0a91f252c4,Splendours of Kerala - Make your own Tea,Luxury,Cochin|Munnar|Thekkady|Allepey,1N Cochin . 2N Munnar . 1N Thekkady . 1N Allepey,Cochin|Munnar|Thekkady|Allepey,13-10-2021,Casino Hotel - Cgh Earth-MMT Holidays Special:...,New Delhi,IndiGo,0,4,Fort Cochin | Dutch Palace | Jewish Synagogue...,Cancellation any time after making the 1st pay...,37128.200257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,75171484ea20440a5c08fe040a9a01a7,Trip to Kerala,Standard,Munnar|Thekkady|Allepey,2N Munnar . 1N Thekkady . 1N Allepey,Munnar|Thekkady|Allepey,04-06-2021,Iceberg Hill Hotel-MMT Holidays Special:4|Elep...,New Delhi,Vistara,2,3,Valara Waterfalls | Tea Plantation at Devikul...,Cancellation any time after making the 1st pay...,28723.166214
8996,16f57cebfb653f2c532314322598ca3a,Magical Kashmir Holiday,Deluxe,Srinagar|Pahalgam|Srinagar|Gulmarg,1N Srinagar . 1N Pahalgam . 1N Srinagar . 1N G...,Srinagar|Pahalgam|Srinagar|Gulmarg,07-03-2021,California Group of Houseboats:Three|The China...,Mumbai,IndiGo,1,3,Dal Lake | Pahalgam | Avantipura Ruins | Beta...,Cancellation any time after making the 1st pay...,28967.124653
8997,477f8b3f07e1a6c027152e2cd5d67e4a,Holiday in Dharamshala and Dalhousie from Delhi,Luxury,Dharamshala|Dalhousie,3N Dharamshala . 2N Dalhousie,Dharamshala|Dalhousie,21-08-2021,Little Mastiff Dharamshala - Unit Of Pong View...,Mumbai,Vistara,0,4,Dalai Lama temple | Bhagsu falls | Church of ...,Cancellation any time after making the 1st pay...,35937.842430
8998,c9ce916ec5e1b93134a2f84090284bf9,"Colorful Rajasthan- Jaipur, Jodhpur, Jaisalmer...",Deluxe,Jaipur|Jodhpur|Jaisalmer|Udaipur,2N Jaipur . 1N Jodhpur . 2N Jaisalmer . 2N Uda...,Jaipur|Jodhpur|Jaisalmer|Udaipur,19-03-2021,Souvenir Peppermint:3.9|Lariya Resort:4.3|Gorb...,New Delhi,IndiGo|IndiGo,1,3,City Palace | Hawa Mahal | Jantar Mantar | Ra...,Cancellation any time after making the 1st pay...,28387.146526
